# WEATHER DATA (crawl thông qua lib và api)

In [5]:
# NOTEBOOK OPEN-METEO
# CELL 1: Khởi tạo môi trường

import pandas as pd
import os
import openmeteo_requests
import requests_cache
from retry_requests import retry

print("Cell 1: Môi trường cho việc thu thập dữ liệu khí tượng đã sẵn sàng.")

Cell 1: Môi trường cho việc thu thập dữ liệu khí tượng đã sẵn sàng.


In [6]:
# NOTEBOOK OPEN-METEO
# CELL 2: Đọc file Metadata

metadata_filename = "../stations_metadata.csv"

if not os.path.exists(metadata_filename):
    print(f"LỖI: File '{metadata_filename}' không tồn tại.")
    print("Vui lòng chạy Cell 2.5 trong notebook OpenAQ trước để tạo file này.")
else:
    df_metadata = pd.read_csv(metadata_filename)
    print(f"Đã đọc thành công thông tin của {len(df_metadata)} trạm từ '{metadata_filename}'.")
    print("\nXem trước thông tin metadata:")
    print(df_metadata.head())

Đã đọc thành công thông tin của 31 trạm từ '../stations_metadata.csv'.

Xem trước thông tin metadata:
   location_id                          name                 start_date  \
0         2539     US Diplomatic Post: Hanoi  2016-01-30 01:00:00+00:00   
1         7441                         Hanoi  2016-11-09 18:00:00+00:00   
2      1285357  SPARTAN - Vietnam Acad. Sci.  2019-11-28 03:00:00+00:00   
3      2161290                      An Khánh  2024-01-29 06:00:00+00:00   
4      2161291                      Cầu Diễn  2024-01-22 01:00:00+00:00   

                    end_date        lat         lon  
0  2016-11-09 16:00:00+00:00  21.021770  105.819002  
1  2025-04-09 15:00:00+00:00  21.021939  105.818806  
2  2020-06-23 14:00:00+00:00  21.047800  105.800000  
3  2025-06-10 03:00:00+00:00  21.002400  105.718100  
4  2024-12-11 14:00:00+00:00  21.039800  105.765200  


In [7]:
# NOTEBOOK OPEN-METEO
# CELL 3

# Kiểm tra xem df_metadata đã được đọc chưa
if 'df_metadata' not in locals():
    print("Vui lòng chạy Cell 2 trước để đọc file metadata.")
else:
    # 1. Thiết lập API client của Open-Meteo
    cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
    retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
    openmeteo = openmeteo_requests.Client(session=retry_session)

    # 2. Tạo thư mục chính để chứa dữ liệu khí tượng
    weather_output_dir = "hanoi_weather_data_raw"
    if not os.path.exists(weather_output_dir):
        os.makedirs(weather_output_dir)
        print(f"Đã tạo thư mục: {weather_output_dir}")

    print(f"\nBắt đầu quá trình tải dữ liệu khí tượng cho {len(df_metadata)} vị trí...")

    # 3. Lặp qua từng trạm trong file metadata
    for index, station in df_metadata.iterrows():
        loc_id = station['location_id']
        lat = station['lat']
        lon = station['lon']
        start_date = station['start_date'][:10]
        end_date = station['end_date'][:10]
        
        print(f"\n--- Đang lấy dữ liệu khí tượng cho vị trí của trạm ID: {loc_id} ---")

        # 4. Xây dựng tham số và gọi API
        url = "https://archive-api.open-meteo.com/v1/archive"
        params = {
            "latitude": lat,
            "longitude": lon,
            "start_date": start_date,
            "end_date": end_date,
            "hourly": ["temperature_2m", "relative_humidity_2m", "precipitation", "rain", "wind_speed_10m", "wind_direction_10m", "pressure_msl"],
            "timezone": "auto"
        }
        
        try:
            responses = openmeteo.weather_api(url, params=params)
            response = responses[0]
            
            # 5. Xử lý dữ liệu trả về và tạo DataFrame
            hourly = response.Hourly()
            
            # --- SỬA LỖI TẠI ĐÂY ---
            # Sử dụng pd.date_range để tạo lại chuỗi thời gian một cách chính xác
            hourly_data = {"datetime": pd.date_range(
                start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
                end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
                freq=pd.Timedelta(seconds=hourly.Interval()),
                inclusive="left"
            )}
            # -------------------------

            variables = params["hourly"]
            for i, var_name in enumerate(variables):
                # Lấy giá trị dưới dạng numpy array
                values = hourly.Variables(i).ValuesAsNumpy()
                # Cắt bớt array nếu nó dài hơn chuỗi thời gian (để phòng trường hợp lỗi)
                hourly_data[var_name] = values[:len(hourly_data["datetime"])]

            df_weather = pd.DataFrame(data=hourly_data)
            df_weather['datetime'] = df_weather['datetime'].dt.tz_convert('UTC')

            # 6. Lưu file CSV
            output_filename = os.path.join(weather_output_dir, f"weather_{loc_id}.csv")
            df_weather.to_csv(output_filename, index=False, encoding='utf-8-sig')
            
            print(f"  -> Đã lấy và lưu thành công {len(df_weather)} giờ dữ liệu khí tượng vào file {output_filename}")

        except Exception as e:
            # In ra lỗi chi tiết hơn
            import traceback
            print(f"  -> Đã xảy ra lỗi khi lấy dữ liệu khí tượng cho vị trí trạm {loc_id}:")
            traceback.print_exc()

    print("\n\n---> HOÀN TẤT QUÁ TRÌNH TẢI DỮ LIỆU KHÍ TƯỢNG <---")
    print(f"Dữ liệu đã được lưu vào các file riêng lẻ trong thư mục '{weather_output_dir}'")


Bắt đầu quá trình tải dữ liệu khí tượng cho 31 vị trí...

--- Đang lấy dữ liệu khí tượng cho vị trí của trạm ID: 2539 ---
  -> Đã lấy và lưu thành công 6840 giờ dữ liệu khí tượng vào file hanoi_weather_data_raw\weather_2539.csv

--- Đang lấy dữ liệu khí tượng cho vị trí của trạm ID: 7441 ---
  -> Đã lấy và lưu thành công 73776 giờ dữ liệu khí tượng vào file hanoi_weather_data_raw\weather_7441.csv

--- Đang lấy dữ liệu khí tượng cho vị trí của trạm ID: 1285357 ---
  -> Đã lấy và lưu thành công 5016 giờ dữ liệu khí tượng vào file hanoi_weather_data_raw\weather_1285357.csv

--- Đang lấy dữ liệu khí tượng cho vị trí của trạm ID: 2161290 ---
  -> Đã lấy và lưu thành công 11976 giờ dữ liệu khí tượng vào file hanoi_weather_data_raw\weather_2161290.csv

--- Đang lấy dữ liệu khí tượng cho vị trí của trạm ID: 2161291 ---
  -> Đã lấy và lưu thành công 7800 giờ dữ liệu khí tượng vào file hanoi_weather_data_raw\weather_2161291.csv

--- Đang lấy dữ liệu khí tượng cho vị trí của trạm ID: 2161292 ---

In [8]:
# NOTEBOOK OPEN-METEO
# CELL 4: Gộp tất cả các file dữ liệu Khí tượng

import pandas as pd
import os
import glob

weather_output_dir = "hanoi_weather_data_raw"

if not os.path.exists(weather_output_dir):
    print(f"Thư mục '{weather_output_dir}' không tồn tại. Vui lòng chạy Cell 3 trước.")
else:
    # 1. Tìm tất cả các file .csv trong thư mục dữ liệu khí tượng
    all_weather_files = glob.glob(os.path.join(weather_output_dir, "weather_*.csv"))
    
    if not all_weather_files:
        print("Không tìm thấy file CSV khí tượng nào để gộp.")
    else:
        print(f"Tìm thấy {len(all_weather_files)} file CSV khí tượng. Bắt đầu gộp...")
        
        weather_dfs = []
        for f in all_weather_files:
            df = pd.read_csv(f)
            # Lấy location_id từ tên file để thêm vào làm một cột
            try:
                loc_id = int(os.path.basename(f).replace('weather_', '').replace('.csv', ''))
                df['location_id'] = loc_id
                weather_dfs.append(df)
            except ValueError:
                print(f"  - Cảnh báo: Không thể lấy location_id từ tên file '{os.path.basename(f)}'. Bỏ qua.")
        
        if weather_dfs:
            # 2. Gộp tất cả các DataFrame lại
            final_df_weather = pd.concat(weather_dfs, ignore_index=True)
            
            # 3. Sắp xếp lại để dễ nhìn và xử lý
            final_df_weather['datetime'] = pd.to_datetime(final_df_weather['datetime'])
            final_df_weather = final_df_weather.sort_values(by=['location_id', 'datetime'])
            
            # Sắp xếp lại thứ tự các cột cho hợp lý
            cols = final_df_weather.columns.tolist()
            cols = ['location_id'] + [col for col in cols if col != 'location_id'] # Đưa location_id lên đầu
            final_df_weather = final_df_weather[cols]
            
            # 4. Lưu file tổng hợp cuối cùng
            final_output_filename = "hanoi_weather_history_COMBINED.csv"
            final_df_weather.to_csv(final_output_filename, index=False, encoding='utf-8-sig')
            
            print("\n---> QUÁ TRÌNH GỘP HOÀN TẤT <---")
            print(f"Tổng cộng có {len(final_df_weather)} dòng dữ liệu khí tượng.")
            print(f"Dữ liệu đã được gộp và lưu vào file '{final_output_filename}'")
        else:
            print("Không có dữ liệu khí tượng hợp lệ để gộp.")

Tìm thấy 31 file CSV khí tượng. Bắt đầu gộp...

---> QUÁ TRÌNH GỘP HOÀN TẤT <---
Tổng cộng có 216720 dòng dữ liệu khí tượng.
Dữ liệu đã được gộp và lưu vào file 'hanoi_weather_history_COMBINED.csv'
